1. Which geographic area of interest (AOI) are you focussing on? This can be the same AOI you selected for written assignment 1, or you can focus on a new AOI.
2. Which of the suggested tools and platforms are the most relevant to your likely applications? And why? 
3. Using your own tools or tools described in the class - can you assess how forests in your AOI changed over the last 20 years?  Has there been a net gain or loss in forests? Can you determine what is responsible for this change? (Hint: Two tools that allow for AOI level analysis without coding are Global Forest Watch and Open Foris Earth Map).
4. Based on IPCC land use land cover (LULC) categories, which specific IPCC categorical transitions do you think are occurring in relation to the forests in your AOI? (Hint: For a quantitative analysis, check out Open Foris Earth Map has an IPCC focused change analysis “IPCC categories based on ESA CCI 1992/2020”)
5. What do you imagine the impacts of this historical change (over the last 20 years) have been on human and wildlife populations?
6. How and why do people use the forests in your AOI?
7. Have carbon emissions reductions and / or removals already been modelled for the forests in your AOI?  If yes: how?  If no: how would you approach the situation?


In [7]:
import ee
import geemap
import geopandas as gpd

In [2]:
ee.Authenticate()
ee.Initialize()

In [9]:
project_gdf = gpd.read_file(
    "../geometries/WDPA_WDOECM_Nov2024_Public_26621_shp/WDPA_WDOECM_Nov2024_Public_26621_shp-polygons.shp"
)
project_fc = ee.FeatureCollection(project_gdf.__geo_interface__)

## Hansen Global Forest Change v1.11 (2000-2023) 

* Forest change over the last 20 years (gains and losses)

In [10]:
hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
hansen = hansen.clip(project_fc)

In [11]:
# Create an animation of forest loss by year

lossyear = hansen.select("lossyear")

In [13]:
lossyear

In [ ]:
# Mask the areas that are not classified as forest by ESA
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')

In [ ]:
loss_ramp = {
    "palette": "Spectral",
    "min": 0,
    "max": 
}

In [16]:
p = geemap.Map()
p.addLayer(lossyear, loss_ramp, "Year of Forest Loss")
p.centerObject(project_fc)
p.add_colorbar(loss_ramp, "Year of Forest Loss")
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Copernicus Global Land Cover Layers: CGLS-LC100 

IPCC has the following Land Use / Land Cover (LULC) definitions:
Forest land, Cropland, Grassland, Wetlands, Settlements, and Other land

To select a LCLU product, the following considerations were used:
* Data availability for at least 2004-2023, on a yearly basis
* As close to IPCC categories as possible, with the possibility of collapsing more granular categories into more general groupings

Products explored:
* The European Space Agency (ESA) WorldCover product had categories that were closest to the IPCC definition, but only 2020 and 2021
* GlobCover: Global Land Cover Map was only available for 2009-2010
* Copernicus Global Land Cover Layers are available from 2015-2019 and have more granular categories than IPCC
* Dynamic World V1 has data from 2015-2024 with categories that align with IPCC
* IPCC Land Use Classification - CCI/ESA. Available 1992-2022. Categories align with IPCC, but does not appear to be available in Google Earth Engine.

For this analysis, we use Copernicus Global Land Cover Layers and translate between classes to the IPCC Land Use Category.

| CCI-LC Class | Description | IPCC Land Use Category |  |
|---|---|---|---|
| 10, 11, 12 | Rainfed cropland | Cropland |  |
| 20 | Irrigated cropland | Cropland |  |
| 30 | Mosaic cropland (>50%) / natural vegetation (<50%) | Cropland |  |
| 40 | Mosaic natural vegetation (>50%) / cropland (<50%) | Cropland |  |
| 50 | Tree cover, broadleaved, evergreen, closed to open (>15%) | Forest |  |
| 60, 61, 62 | Tree cover, broadleaved, deciduous, closed to open (>15%) | Forest |  |
| 70, 71, 72 | Tree cover, needleleaved, evergreen, closed to open (>15%) | Forest |  |
| 80, 81, 82 | Tree cover, needleleaved, deciduous, closed to open (>15%) | Forest |  |
| 90 | Tree cover, mixed leaf type (broadleaved and needleleaved) | Forest |  |
| 100 | Mosaic tree and shrub (>50%) / herbaceous cover (<50%) | Forest |  |
| 110 | Mosaic herbaceous cover (>50%) / tree and shrub (<50%) | Grassland |  |
| 120, 121, 122 | Shrubland | Grassland |  |
| 130 | Grassland | Grassland |  |
| 140 | Lichens and mosses | Grassland |  |
| 150, 151, 152, 153 | Sparse vegetation (tree, shrub, herbaceous cover) | Grassland |  |
| 160 | Tree cover, flooded, fresh or brakish water | Wetland |  |
| 170 | Tree cover, flooded, saline water | Wetland |  |
| 200, 201, 202 | Bare areas | Otherland |  |
| 210 | Water Bodies | No data |  |
| 220 | Permanent snow and ice | Otherland |  |

In [ ]:
land_cover_collection = ee.Image(
    "COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019"
).select("discrete_classification")
land_cover_collection

## Biomass Estimation

* Global Aboveground and Belowground Biomass Carbon Density Maps from NASA ORNL DAAC at Oak Ridge National Laboratory was produced for 2010 (Aboveground living biomass carbon stock and belowground carbon stock as separate bands) at 300 meter resolution. This product also provides an uncertainty band for both AGB and BGB.
*  WCMC Above and Below Ground Biomass Carbon Density is available in tropical countries, also produced in the year 2010 (Tonnes of above and below ground biomass carbon per hectare as a single band) at 300 meter resolution
*  WHRC Pantropical National Level Carbon Stock Dataset was produced for the year 2012 (Megagrams of aboveground live woody biomass per Hectare) at 500m resolution
  
The  Global Aboveground and Belowground Biomass Carbon Density product was chosen for this analysis because it provides uncertainty bands and estimates for AGB and BGB as separate bands.

In [ ]:
biomass = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")